# Toxicity through time

In [ ]:
%reload_ext autoreload
%autoreload 2

import os 
import sys
import pandas as pd
import numpy as np
import plotly 
import plotly.graph_objects as go
import time

import nltk
from detoxify import Detoxify
# nltk.download('stopwords')

try:
    print(run_only_once)
except Exception as e:
    print(os.getcwd())
    os.chdir("./../../")
    print(os.getcwd())
    run_only_once = "Dir has already been changed"

Toxicity metrics through time plotted.

## Most and least toxic tweets in our data set

More preprocessing has to be done on tweets to improve the accuracy of the toxicity prediction model because this shit is hillarious.

## Hashtag toxicity comparison

In [ ]:
def compare_hashtag_toxicity(files_list):
    comparison_list = []
    tweet_count = 0
    for file_name in files_list:
        hashtag_name = file_name.split("_")[0]
        print(f"For hashtag {hashtag_name}, from file {file_name}")

        file_name = f"./data/preprocessed/{file_name}"
        tweets_df = pd.read_csv(file_name)
        # display(tweets_df)
        tweet_count += len(tweets_df.index)
        tweets_df["created_at"] = pd.to_datetime(tweets_df["created_at"])
        # group by date 
        toxicity_cols = ["toxicity", "severe_toxicity", "obscene", "threat", "insult", "identity_attack"]
        toxicity_cols = [f"normal_{i}" for i in toxicity_cols]
        selected_cols = ["retweet_count", "reply_count", "like_count", "quote_count", "created_at"] + toxicity_cols
        grouped_by_day = tweets_df[selected_cols].resample("D", on="created_at").mean()

        # go.Scatter(x=grouped_by_day.index,y=grouped_by_day["toxicity"], name=hashtag_name, mode="lines")
        comparison_list.append(tweets_df)
        
        """
        # uncomment for single plots
        col_traces = []
        for column in toxicity_cols:
            col_traces.append(
                go.Scatter(x=grouped_by_day.index,y=grouped_by_day[column], name=column, mode="lines"))
                
        fig = go.Figure(col_traces)
        fig.update_layout(
            title_text="Change of toxicity metrics through time", 
            yaxis_title="Date",
            xaxis_title="Mean metric value",
        )
        fig.update_layout(bargap=0.2)
        fig.show()
        """
        
    """
    fig_com = go.Figure(comparison_list)
    fig_com.update_layout(
        title_text="Change of toxicity through time for each hashtag", 
        yaxis_title="Date",
        xaxis_title="Mean value",
    )
    fig_com.update_layout(bargap=0.2)
    fig_com.show()
    """
    print(f"There is {tweet_count} tweets in all files together!")
    return comparison_list

In [ ]:
# get all the files -> os.listdir("../../data/detoxify_toxicity_added_hashtags/")
#               "trump_hashtag_04_01_2023_detoxify_toxicity.csv",
# print(os.listdir("./data/preprocessed/"))
"""
files_list = ["fitness_hashtag_08_01_2023_detoxify_toxicity.csv",
              "musk_hashtag_03_01_2023_detoxify_toxicity.csv",
              "netflix_hashtag_08_01_2023_detoxify_toxicity.csv",
              "uno_hashtag_09_01_2023_detoxify_toxicity.csv",
              "vegan_hashtag_6_1_2023_detoxify_toxicity.csv",
              "vegetarian_hashtag_6_1_2023_detoxify_toxicity.csv"]
"""
files_list = ['fitness_hashtag_merged.csv', 'musk_hashtag_merged.csv', 'netflix_hashtag_merged.csv', 
 'trump_hashtag_merged.csv', 'uno_hashtag_merged.csv', 'vegan_hashtag_merged.csv', 
 'vegetarian_hashtag_merged.csv']

df_array = compare_hashtag_toxicity(files_list)

In [ ]:
len_hashtag_list = []
for file_name, df_i in zip(files_list, df_array):
    hashtag = file_name.split("_")[0]
    len_hashtag_list.append((hashtag, len(df_i.index)))
len_hashtag_list = sorted(len_hashtag_list, key=lambda tup: tup[1], reverse=True)    
    
"""fig = go.Figure(comparison_list)
fig.update_layout(
    title_text="Change of toxicity through time for each hashtag", 
    yaxis_title="Date",
    xaxis_title="Mean value",
)
fig.update_layout(bargap=0.2)
fig.show()"""
fig = go.Figure([go.Bar(x=[i[0] for i in len_hashtag_list], y=[i[1] for i in len_hashtag_list])])
fig.update_layout(
    title_text="Hashtag count in period from 01/06/2022 to 03/01/2023", 
    yaxis_title="Count",
    xaxis_title="Hashtag",
)
fig.show()

len_hashtag_list

In [ ]:
mean_list = []
median_list = []
keep_columns = ['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult','identity_attack']
keep_columns = [f"normal_{i}" for i in keep_columns]
for file_name, df_i in zip(files_list, df_array):
    hashtag = file_name.split("_")[0]
    # create a list of all metrics
    tmp_list_median = [hashtag] + df_i.drop('Unnamed: 0', axis=1)[keep_columns].median().tolist()
    tmp_list_mean = [hashtag] + df_i.drop('Unnamed: 0', axis=1)[keep_columns].mean().tolist()
    mean_list.append(tmp_list_mean)
    median_list.append(tmp_list_median)

In [ ]:
toxicity_mean_df = pd.DataFrame(mean_list, columns=["hashtag"] + keep_columns).sort_values("normal_toxicity", ascending=False)
toxicity_median_df = pd.DataFrame(median_list, columns=["hashtag"] + keep_columns).sort_values("normal_toxicity", ascending=False)

In [ ]:
fig = go.Figure([go.Bar(x=toxicity_median_df["hashtag"], y=toxicity_median_df["normal_toxicity"])])
fig.update_layout(
    title_text="Median toxicity per hashtag from 01/06/2022 to 03/01/2023", 
    yaxis_title="Count",
    xaxis_title="Hashtag",
)
fig.show()

fig = go.Figure([go.Bar(x=toxicity_mean_df["hashtag"], y=toxicity_mean_df["normal_toxicity"])])
fig.update_layout(
    title_text="Mean toxicity per hashtag from 01/06/2022 to 03/01/2023", 
    yaxis_title="Count",
    xaxis_title="Hashtag",
)
fig.show()

In [ ]:
groped_by_day_list = []
scatter_list = []
for file_name, df_i in zip(files_list, df_array):
    hashtag = file_name.split("_")[0]
    df_i["created_at"] = pd.to_datetime(df_i["created_at"])
    
    # group by date 
    toxicity_cols = ["toxicity", "severe_toxicity", "obscene", "threat", "insult", "identity_attack"]
    toxicity_cols = [f"normal_{i}" for i in toxicity_cols]
    selected_cols = ["retweet_count", "reply_count", "like_count", "quote_count", "created_at"] + toxicity_cols
    grouped_by_day = df_i[selected_cols].resample("D", on="created_at").mean()
    scatter_list.append(go.Scatter(x=grouped_by_day.index,y=grouped_by_day["normal_toxicity"], name=hashtag, mode="lines"))
    groped_by_day_list.append(grouped_by_day[toxicity_cols])

In [ ]:
fig_com = go.Figure(scatter_list)
fig_com.update_layout(
    title_text="Change of toxicity through time for each hashtag", 
    yaxis_title="Mean value",
    xaxis_title="Date",
)
fig_com.update_layout(bargap=0.2)
fig_com.show()

In [ ]:
concat_df = pd.concat(groped_by_day_list) 
merged_tox_cols = concat_df.groupby(concat_df.index).mean()
display(merged_tox_cols)

In [ ]:
toxicity_cols = ["toxicity", "severe_toxicity", "obscene", "threat", "insult", "identity_attack"]
toxicity_cols = [f"normal_{i}" for i in toxicity_cols]

metrics_list = []
for col in toxicity_cols:
    metrics_list.append(go.Scatter(x=merged_tox_cols.index,y=merged_tox_cols[col], name=col, mode="lines"))
    
fig_com = go.Figure(metrics_list)
fig_com.update_layout(
    title_text="Change of toxicity metrics through time for all hashtags", 
    yaxis_title="Mean value",
    xaxis_title="Date",
)
fig_com.update_layout(bargap=0.2)
fig_com.show()

In [ ]:
pd.set_option('display.max_colwidth', None)
df_array[0].drop("Unnamed: 0", axis=1).nlargest(20, "normal_toxicity")[["tweet_id", "text", "normal_toxicity"]]

In [ ]:
pd.set_option('display.max_colwidth', None)
df_array[0].drop("Unnamed: 0", axis=1).nsmallest(5, "normal_toxicity")[["tweet_id", "text", "normal_toxicity"]]